In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [48]:
# Import data
import numpy as np
import pandas as pd
np.random.seed(4705)

data = pd.read_csv('/content/gdrive/My Drive/NLP/stance-data.csv')
data.columns

Index(['post_text', 'topic', 'author', 'label', 'id', 'word_count',
       'words_pronom', 'words_per_sen', 'words_over_6', 'pos_emo', 'neg_emo',
       'count_noun', 'count_verb', 'count_adj'],
      dtype='object')

In [0]:
# Select topic and shuffle data
topic = "abortion"
selected = data[data.topic==topic]
selected = np.random.permutation(selected)
#set(selected[:,1])

In [0]:
# Import functions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV

# 2. Method 1 - Ngrams (no idf)

In [12]:
# Show that ngram_range don't need to be params because of feature selection 
# cv = CountVectorizer(stop_words='english', ngram_range=(1,4))
# print(np.shape(cv.fit_transform(selected[:,0])))
# cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
# print(np.shape(cv.fit_transform(selected[:,0])))
# cv = CountVectorizer(stop_words='english', ngram_range=(3,4))
# print(np.shape(cv.fit_transform(selected[:,0])))

(1708, 237620)
(1708, 74275)
(1708, 163345)


In [16]:
np.shape(y)

(1708,)

In [0]:
# Set label as y
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected[:,0])

In [7]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000,2000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6077309598532009
clf__alpha: 1
exf__k: 500


In [8]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6135789130695752
clf__alpha: 0.5
exf__k: 50


In [10]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6141877175833033
clf__alpha: 1
exf__k: 100


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)

In [25]:
print(gs.best_score_, gs.best_params_)

0.6071375898201026 {'clf__C': 100, 'clf__gamma': 0.001, 'clf__kernel': 'rbf', 'exf__k': 100}


## 找到abortion最好组合(Ngram)：0.614
0.6141877175833033
clf__alpha: 1
exf__k: 100

ComplementNB


## Gay rights

In [0]:
# Select topic and shuffle data
topic2 = "gay rights"
selected2 = data[data.topic==topic2]
selected2 = np.random.permutation(selected2)
#set(selected2[:,1])
# Set label as y
y = selected2[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected2[:,0])

In [30]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000,2000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6210507819363908
clf__alpha: 1
exf__k: 20


In [31]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6166119243610018
clf__alpha: 1
exf__k: 1000


In [32]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5791249341064839
clf__alpha: 1
exf__k: 500


In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X, y)

In [34]:
print(gs.best_score_, gs.best_params_)

0.6453401457091494 {'clf__C': 100, 'clf__gamma': 0.0001, 'clf__kernel': 'rbf', 'exf__k': 20}


## 找到gay rights最好组合（Ngram）:0.645, 'clf__C': 100, 'clf__gamma': 0.0001, 'clf__kernel': 'rbf', 'exf__k': 20, SVC

# 3. Method 2-Other features (with idf?)

## Abortion,先不用idf

In [35]:
set(selected[:,1])

{'abortion'}

In [0]:
# Extract features
# Select topic and shuffle data
# Set label as y
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,4))
X = X.fit_transform(selected[:,0])
#idf???????

from scipy.sparse import hstack
X_more = hstack((X, selected[:,5:].astype(float)))

In [37]:
# Build pipeline with MultinomialNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5842842688343537
clf__alpha: 1
exf__k: 1000


In [40]:
# Build pipeline with BernoulliNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', BernoulliNB()),
])
params = {
    'exf__k':(20,50,100,500,1000),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.6065596542676339
clf__alpha: 1
exf__k: 1000


In [41]:
# Build pipeline with ComplementNB as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', ComplementNB()),
])
params = {
    'exf__k':(20,50,100,500),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

0.5690761605871962
clf__alpha: 0.1
exf__k: 500


In [43]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC()),
])
params = [  
    {'exf__k':[20,50,100,500],'clf__kernel':['rbf'],'clf__gamma': [1e-3, 1e-4],
     'clf__C': [0.1,1, 10, 100]},
    {'exf__k':[20,50,100,500],'clf__kernel': ['linear'], 'clf__C': [0.1,1, 10, 100]},
]
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)

KeyboardInterrupt: ignored

In [0]:
# Build pipeline with SVC as classifier
pipe = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SVC(kernel='rbf')),
])
params = {'exf__k':[20,50,100,500],
          'clf__C': [0.1,1, 10, 100],
          'clf__gamma': [1e-3, 1e-4],
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(X_more, y)

In [45]:
print(gs.best_score_, gs.best_params_)

0.6007031263397986 {'clf__C': 100, 'clf__gamma': 0.0001, 'exf__k': 500}


## 试试SGDClassifier->loss=hinge是SVM

In [47]:
from sklearn.linear_model import SGDClassifier
SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier(loss='hinge')),
])
paras_SVM = {
    'exf__k':[20,50,100,500,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__max_iter':(1000,2000,5000),
    
}
gs_clf_SVM = GridSearchCV(SGD, paras_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(paras_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6106292123270052
clf__alpha: 0.01
clf__max_iter: 5000
clf__penalty: 'None'
exf__k: 1000


In [54]:

#2->5,4->7,6->9
X_more = hstack((X, selected[:,7:9].astype(float)))
SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier()),
])
parameters_SVM = {
    'exf__k':[20,50,100,500,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__loss':('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
    'clf__max_iter':(1000,5000,10000,20000),
    
}
gs_clf_SVM = GridSearchCV(SGD, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

KeyboardInterrupt: ignored

## 对other feature, svc换别的参数看看1.+idf, 2.只考虑6个liwc

In [49]:
y = selected[:,3]

# Extract features
X = CountVectorizer(stop_words='english', ngram_range=(1,3))
X = X.fit_transform(selected[:,0])

from scipy.sparse import hstack
X_more = hstack((X, selected[:,5:8].astype(float)))

from sklearn.linear_model import SGDClassifier
SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier(loss='hinge')),
])
paras_SVM = {
    'exf__k':[20,50,100,500,800,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__max_iter':(1000,2000,5000),
    
}
gs_clf_SVM = GridSearchCV(SGD, paras_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(paras_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6153127197269812
clf__alpha: 0.001
clf__max_iter: 1000
clf__penalty: 'l1'
exf__k: 800


In [52]:
X_more = hstack((X, selected[:,5:11].astype(float)))

SGD = Pipeline([
    ('exf', SelectKBest(chi2)),
    ('clf', SGDClassifier(loss='hinge')),
])
paras_SVM = {
    'exf__k':[20,50,100,500,800,1000,2000],
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__max_iter':(1000,2000,5000),
    
}
gs_clf_SVM = GridSearchCV(SGD, paras_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_more, y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(paras_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.612405892541716
clf__alpha: 0.001
clf__max_iter: 1000
clf__penalty: 'l1'
exf__k: 2000


#其实加了后面几个feature没啥用

In [51]:
np.shape(selected[:,5:11])

(1708, 6)

# Advanced Pipeline for Method2

In [55]:
np.shape(selected)

(1708, 14)

In [0]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import FeatureUnion
class AddLiwc(TransformerMixin):
  def transform(self, X, **transform_params):
    return DictVectorizer(X[:, 5:11])
  
  def fit(self, X, y=None, **fit_params):
    return self
  
class GetText(TransformerMixin):
  def transform(self, X, **transform_params):
    return X[:, 0]
  
  def fit(self, X, y=None, **fit_params):
    return self  

In [0]:
pipe = Pipeline([
    ('features', FeatureUnion([
       ('ngram_tf_idf',Pipeline([
           ('txt', GetText()),
           ('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))),
           #('tfidf', TfidfTransformer())
       ])), 
       ('liwc', AddLiwc())
    ])),
    ('exf', SelectKBest(chi2)),
    ('clf', MultinomialNB())    
])
params = {
  #'tfidf__use_idf': [False],
  'exf__k':[20],
  'clf__alpha':[1],
#   'tfidf__use_idf': (True, False),
#   'exf__k':(20,50,100,500,1000,2000),
#   'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}

In [0]:
from sklearn.feature_extraction import DictVectorizer
cv = CountVectorizer(stop_words='english', ngram_range=(1,3))
t1 = cv.fit_transform(selected[:,0])
t2 = DictVectorizer(selected[:,-1])

In [93]:
FeatureUnion([])

TypeError: ignored

In [90]:
pipe = Pipeline([
    ('features', FeatureUnion([
       ('ngram_tf_idf',Pipeline([
           ('txt', GetText()),
           ('vect', CountVectorizer(stop_words='english', ngram_range=(1,3)))
       ])), 
       ('liwc', AddLiwc())
    ])),
    ('clf', MultinomialNB())    
])
pipe.fit(selected[:,:-1], selected[:,3])

ValueError: ignored

In [80]:
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(selected[:,:-1], selected[:,3])
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

TypeError: ignored

In [0]:
pipe = Pipeline([
  ('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))),
  ('exf', SelectKBest(chi2)),
  ('clf', MultinomialNB()),
])
params = {
  'exf__k':(20,50,100,500,1000,2000),
  'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs = GridSearchCV(pipe, params, cv=5, iid=False, n_jobs=-1)
gs = gs.fit(selected[:,0], selected[:,3])
print(gs.best_score_)
for param_name in sorted(params.keys()):
  print("%s: %r" % (param_name, gs.best_params_[param_name]))

In [56]:
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

# My search
每一组参数，都得到平均的CV结果，然后再从平均值中选最大的。

所以可以是search调用run

run中分train 和test.

In [101]:
params_NB = {
    'exf_k':(20,50,100,500,1000,2000),
    'clf_alpha':(1,0.5,1e-1,1e-2,1e-3)
}
comb_NB = []
for i in range(len(params_NB['exf_k'])):
  for j in range(len(params_NB['clf_alpha'])):
    comb_NB.append((params_NB['exf_k'][i], params_NB['clf_alpha'][j]))
print(comb_NB[:6])

[(20, 1), (20, 0.5), (20, 0.1), (20, 0.01), (20, 0.001), (50, 1)]


In [0]:
def run(selected, paras, method=1, NB=True):
  kfolds = 5
  test_size = len(selected)//kfolds
  cv = CountVectorizer(stop_words='english', ngram_range=(1,3))
  exf_k = paras[0]
  kBest = SelectKBest(chi2, k=exf_k)

  for i in range(kfolds):
    test = selected[i*test_size:(i+1)*test_size]
    train = np.vstack((selected[:i*test_size,:],selected[(i+1)*test_size:,:]))
    
    trainY = train[:,3]
    testY = test[:,3]

    trainX = cv.fit_transform(train[:,0],trainY)
    
    
    trainX = kBest.fit_transform(trainX, trainY)
    testX = cv.transform(test[:,0])
    testX = kBest.transform(testX)

In [0]:
def MySearch(selected):
  kfolds = 5
  test_size = len(selected)//kfolds
  cv = CountVectorizer(stop_words='english', ngram_range=(1,3))
  kBest = SelectKBest(chi2, k=k)

  
  for i in range(kfolds):
  test = selected[i*test_size:(i+1)*test_size]
  train = np.vstack((selected[:i*test_size,:],selected[(i+1)*test_size:,:]))
  print('fold: %d, train size: %d, test size: %d' %(i,len(train),len(test)))

  trainY = train[:,3]
  testY = test[:,3]

  trainX = cv.fit_transform(train[:,0],trainY)
  trainX = kBest.fit_transform(trainX, trainY)
  testX = cv.transform(test[:,0])
  testX = kBest.transform(testX)

  clf.fit(trainX, trainY)
  pred = clf.predict(testX)
  score += metrics.accuracy_score(testY, pred)
  #average in f1 could be any of 'macro','micro','weighted'
  f1 += metrics.f1_score(testY, pred, average='micro')
  print("The averaged accuracy score of 5-fold cv is %0.2f and f1 score is %0.2f." %(score/5, f1/5))

In [97]:
list(params_NB.keys())[0]
Nparas = len(parameters)
for ipara in range(Nparas):
  

'k'